# Improving Fast Minimum-Norm Attacks with Hyperparameter Optimization

In [9]:
import math
import torch

from models.load_data import load_data
from tuning.tune_fmn import tune_fmn

device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

ValueError: mutable default <class 'ray.data._internal.execution.interfaces.ExecutionResources'> for field resource_limits is not allowed: use default_factory

In [6]:
MODELS = {
    0: 'Wang2023Better_WRN-70-16',
    1: 'Wang2023Better_WRN-28-10',
    2: 'Gowal2021Improving_70_16_ddpm_100m',
    3: 'Rebuffi2021Fixing_106_16_cutmix_ddpm',
    4: 'Gowal2021Improving_28_10_ddpm_100m',
    5: 'Pang2022Robustness_WRN70_16',
    6: 'Sehwag2021Proxy_ResNest152',
    7: 'Pang2022Robustness_WRN28_10',
    8: 'Gowal2021Improving_R18_ddpm_100m'
}

OPTIMIZERS = {
    0: "SGD",
    1: "SGDNesterov",
    2: "Adam",
    3: "AdamAmsgrad"
}

SCHEDULERS = {
    0: "CosineAnnealingLR",
    1: "CosineAnnealingWarmRestarts",
    2: "MultiStepLR",
    3: "ReduceLROnPlateau"
}

LOSSES = {
    0: 'LL',    # Logit Loss
    1: 'CE'     # Cross Entropy Loss
}

In [7]:
model_id = 8
batch_size = 50
dataset_percent = 0.5

# dataset: Cifar10, norm: LInf
model, dataset = load_data(model_id)

# load a subset of Cifar10 for tuning the attack
dataset_frac = list(range(0, math.floor(len(dataset)*dataset_percent)))
dataset_frac = torch.utils.data.Subset(dataset, dataset_frac)

data_loader = torch.utils.data.DataLoader(dataset_frac,
                                          batch_size=batch_size,
                                          shuffle=False)

Files already downloaded and verified


## Compute clean accuracy
Testing the model with the dataset used for tuning FMN

In [8]:
total, correct = 0.0, 0.0
for b, (inputs, labels) in enumerate(data_loader):
    inputs, labels = inputs.to(device), labels.to(device)
    pred = model(inputs).argmax(1)
    total += labels.shape[0]
    correct += (pred == labels).sum()
    if total >= 1000:
        break

print("Model name: {}".format(MODELS[model_id]))
print("Clean accuracy: {:.4f}".format(correct / total))

Model name: Gowal2021Improving_R18_ddpm_100m
Clean accuracy: 0.8810


## Tune FMN HyperParameters

In [9]:
optimizer = OPTIMIZERS[0]   # SGD
scheduler = SCHEDULERS[0]   # CosineAnnealingLR
steps = 10
epochs = 1
num_samples = 2
loss = LOSSES[0]            # LL

fmn_best_config = tune_fmn(model=model,
                           data_loader=data_loader,
                           optimizer=optimizer,
                           scheduler=scheduler,
                           batch=batch_size,
                           steps=steps,
                           epochs=epochs,
                           num_samples=num_samples,
                           loss=loss)

NameError: name 'tune_fmn' is not defined